In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import vertexai
from vertexai.generative_models import GenerativeModel,

import pprint

# Initialize Vertex AI
vertexai.init(project='with-comment-418817')
# Load the model
model = GenerativeModel("gemini-1.0-pro")

def generate_text(model, prompt, config=None) -> str:
  # Query the model
  response = model.generate_content(prompt, generation_config=config)
  return response


In [14]:
import json
def parse_to_list(s: str) -> dict:
  s = s.replace('\n', '').replace('\\', '')
  i = s.index('[')
  j = s.index(']')
  s = s[i:j + 1]
  try:
    return json.loads(s)
  except Exception as e:
    print(s)
    raise e
  
def parse_to_dict(s: str) -> dict:
  s = s.replace('\n', '').replace('\\', '')
  i = s.index('{')
  j = s.index('}')
  s = s[i:j + 1]
  try:
    return json.loads(s)
  except Exception as e:
    print(s)
    raise e

In [16]:
import configs, prompts

location = 'Ontario'
category = 'Healthcare'

charities = generate_text(
  model,
  prompts.list_charities.format(location, category),
  configs.deterministic
).text

charities = parse_to_list(charities)
pprint.pprint(charities)

[{'Charity name': 'SickKids Foundation',
  'FinancialTransparency': 8,
  'FundraisingTechnique': 'The SickKids Foundation raises funds through a '
                          'variety of channels, including major gifts, special '
                          'events, and direct mail.',
  'Impact': "SickKids is one of the world's foremost paediatric health-care "
            'institutions and the largest centre dedicated to improving '
            "children's health in Canada.",
  'Mission': 'To inspire our community to invest in health and well-being for '
             'children in Canada and around the world.',
  'PublicPerception': 9},
 {'Charity name': 'The Hospital for Sick Children (SickKids)',
  'FinancialTransparency': 8,
  'FundraisingTechnique': 'The Hospital for Sick Children raises funds through '
                          'a variety of channels, including major gifts, '
                          'special events, and direct mail.',
  'Impact': "SickKids is one of the world's fore

In [18]:
output = dict()
for charity in charities:
  name = next(iter(charity.values()))
  detail = dict(list(charity.items())[1:])
  spending_breakdown = generate_text(
    model,
    prompts.spending_breakdown.format(charity),
    configs.deterministic
  ).text

  spending_breakdown = parse_to_dict(spending_breakdown)

  output[name] = {'detail': detail, 'spending': spending_breakdown}

pprint.pprint(output)

{'Princess Margaret Cancer Foundation': {'detail': {'FinancialTransparency': 7,
                                                    'FundraisingTechnique': 'The '
                                                                            'Princess '
                                                                            'Margaret '
                                                                            'Cancer '
                                                                            'Foundation '
                                                                            'raises '
                                                                            'funds '
                                                                            'through '
                                                                            'a '
                                                                            'variety '
                                                           

In [50]:
output

{'SickKids Foundation': {'detail': {'Mission': 'To inspire our community to invest in health and well-being for children in Canada and around the world.',
   'Public perceptions': 9,
   'Impact': 'SickKids Foundation has funded over $1.3 billion in research, innovation, and patient care at SickKids Hospital.',
   'Fundraising Technique': 'SickKids Foundation uses a variety of fundraising techniques, including major gifts, special events, and online donations.',
   'Financial transparency': "SickKids Foundation is a registered charity with the Canada Revenue Agency and is subject to its financial reporting requirements. The foundation's financial statements are audited annually by an independent accounting firm."},
  'spending': {'Program Expenses': 0.85,
   'Fundraising Expenses': 0.1,
   'Administration Expenses': 0.03,
   'Capital Expenditures': 0.01,
   'Other Expenses': 0.01}},
 'Princess Margaret Cancer Foundation': {'detail': {'Mission': 'To fund innovative cancer research, provi